In [2]:
pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [23]:
import pandas as pd
from scipy.stats import pearsonr
from sentence_transformers import SentenceTransformer, util
from torch.utils.data import Dataset
from sklearn.preprocessing import MinMaxScaler
import math

In [31]:
class CustomDataset(Dataset):
    def __init__(self, file_path, scaler=None):
        self.data = pd.read_csv(file_path, sep='\t').dropna()
        self.scaler = scaler

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        score = item[0]
        sentence1 = str(item[1])
        sentence2 = str(item[2])
        if self.scaler:
            score = self.scaler.transform([[score]])[0][0]
        return sentence1, sentence2, score

def map_cos_to_zero_one(x):
    mapped_value = (x + 1) / 2
    return mapped_value

scaler = MinMaxScaler(feature_range=(0, 1)) 
validation_data = CustomDataset('Data/dev.csv', scaler=scaler)

scores = validation_data.data['score'].values.reshape(-1, 1)
scaler.fit(scores)

model = SentenceTransformer("all-MiniLM-L6-v2")

cos_similarities = []
labels = []

for sentence1, sentence2, score in validation_data:
    embedding1 = model.encode(sentence1, convert_to_tensor=True)
    embedding2 = model.encode(sentence2, convert_to_tensor=True)

    cosine_similarity = util.pytorch_cos_sim(embedding1, embedding2).item()
    mapped_value = map_cos_to_zero_one(cosine_similarity)

    cos_similarities.append(mapped_value)
    labels.append(score)

pearsons_corr, _ = pearsonr(labels, cos_similarities)

print("Pearson's correlation coefficient on validation set:", pearsons_corr)


Pearson's correlation coefficient on validation set: 0.8631423871595579
